In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,mean_absolute_error
#plt.rcParams["font.family"] = 'Arial Unicode MS'
#plt.rcParams['axes.unicode_minus'] = False

In [17]:
traing=pd.read_excel('样本数据.xlsx',index_col=0)
traing.head()

,原料换热器管程总管进口温度,1.0MPa蒸汽进装置温度,氮气进装置流量,稳定塔顶压力,K101机出口压力,蒸汽进装置流量,加氢裂化轻石脑油进装置流量,辛烷值1RON,"硫含量2,μg/g"
0,55.613933,196.993645,476.208445,0.652138,3.131928,1458.339950,6368.74700,90.6,3.2
1,56.461392,197.485060,453.687235,0.652110,3.131915,1142.179200,6360.64530,90.5,3.2
2,54.698909,201.774005,454.138060,0.652395,3.129382,1025.874790,6504.96490,90.7,3.2
3,57.883680,196.421215,467.951720,0.652089,3.130650,967.852340,6506.82495,90.4,3.2
4,57.303791,192.642635,470.470510,0.652213,3.128421,932.608515,6560.24225,89.6,3.2


In [18]:
#变量取值范围
interval=pd.read_excel('位号最大值最小值.xlsx')
interval

,位号,最小值,最大值
0,原料换热器管程总管进口温度,40.00,80.00
1,1.0MPa蒸汽进装置温度,150.00,250.00
2,氮气进装置流量,250.00,800.00
3,稳定塔顶压力,0.60,0.70
4,K101机出口压力,2.55,3.55
5,蒸汽进装置流量,0.00,3500.00
6,加氢裂化轻石脑油进装置流量,0.00,12000.00


In [19]:
#读取因变量
y=pd.read_excel('/Users/zhongrunping/Desktop/jupyter文件夹/数学建模/RON损失(不是变量).xlsx')[::-1].reset_index(drop=True)
y=y.iloc[:,2]

In [20]:
#训练模型
svr=SVR(kernel='poly',coef0=5,degree=3,cache_size=800).fit(traing,y)
mean_absolute_error(y,svr.predict(traing))
#训练平均的绝对误差0.15

0.15308364807468453

In [21]:
data=traing.copy()
data=data[154:254]
#硫含量大于5的 设置成5
data['硫含量2,μg/g']=data['硫含量2,μg/g'].apply(lambda x:5 if x>5 else x)
data=data.reset_index(drop=True)
len(data)

100

In [22]:
#获取变量的取值范围
def get_column_range(column):
    index=interval['位号'][interval['位号']==column].index[0]
    maximum=interval['最大值'][index]
    minimum=interval['最小值'][index]
    return minimum,maximum  #返回最大最小值

In [23]:
#定义随机生成一个新解的函数
def generate_new_x(x_old,columns,t):
    a=np.random.randn(len(x_old)-2)
    b=a/((sum(a**2))**0.5)
    x_new=[]
    for i,j in enumerate(b):
        new=x_old[i]+t*j
        low,up=get_column_range(columns[i])
        if new>=low and new<=up:
            x_new.append(new)
        elif new<low:
            while new<low:
                r=np.random.uniform(0,1)
                new=r*low+(1-r)*new
            x_new.append(new)
        elif new>up:
            while new>up:
                r=np.random.uniform(0,1)
                new=r*low+(1-r)*new
            x_new.append(new)
    x_new.append(x_old[-2])
    x_new.append(x_old[-1])
    return x_new

In [24]:
#生成个样本的最初始解
initial_array=[]
for j in range(len(data)):
    initial_x=[]
    for i in range(7):
        a,b=get_column_range(traing.columns[i])
        initial_x.append(np.random.uniform(a,b))
    initial_x.append(data.iloc[j,-2])
    initial_x.append(data.iloc[j,-1])    
    initial_array.append(initial_x)
len(initial_array)

100

In [25]:
result=pd.DataFrame()
for index in range(len(data)):
    x=initial_array[index]
    for t in range(200):
        for i in range(100):
            f_A=svr.predict(np.array(x).reshape(1,-1))
            new_x=generate_new_x(x,data.columns,t)
            f_B=svr.predict(np.array(new_x).reshape(1,-1))
            if f_B<=f_A:
                x=new_x
                #print('当前新解小于旧解，将新的解赋给x')
            elif f_B>f_A:
                delta_f=abs(f_B-f_A)
                p=np.exp(-(delta_f/(100*(0.95**t))))
                r=np.random.uniform(0,1)
                if p>r:
                    x=new_x
                    #print('接受新解的概率为{},随机概率为：{},接受新解。'.format(p[0],r))
    result=pd.concat([result,pd.DataFrame(np.array(x).reshape(1,-1))],axis=0)
    result.to_excel('154-254.xlsx')